In [1]:
! git clone https://github.com/vishal-burman/PyTorch-Architectures.git

fatal: destination path 'PyTorch-Architectures' already exists and is not an empty directory.


In [1]:
%cd PyTorch-Architectures/modeling_openai/

/content/PyTorch-Architectures/modeling_openai


In [ ]:
! pip install transformers
! pip install ftfy
! pip install spacy
! pip install datasets

In [20]:
import time
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from transformers import OpenAIGPTTokenizer
from model import OpenAIGPTLMHeadModel
from config_openai import OpenAIGPTConfig
config = OpenAIGPTConfig()
tokenizer = OpenAIGPTTokenizer.from_pretrained("openai-gpt")
# pad_token is not set by default
tokenizer.pad_token = '[PAD]'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
from datasets import load_dataset
dataset = load_dataset('cnn_dailymail', '3.0.0')

In [4]:
##############################################
# PyTorch Dataset
class CustomDataset(Dataset):

    def __init__(self, texts, tokenizer):
        self.texts = texts
        self.tokenizer = tokenizer
        self.list_texts = []
        self.build()

    def __len__(self):
        return len(self.list_texts)

    def __getitem__(self, index):
        ids = self.list_texts[index]['input_ids']
        mask = self.list_texts[index]['attention_mask']
        
        return{
                'ids': torch.tensor(ids, dtype=torch.long),
                'mask': torch.tensor(mask, dtype=torch.long)
                }

    def build(self):
        for text in self.texts:
            self.list_texts.append(self.tokenizer(text, max_length=32, pad_to_max_length=True, truncation=True))
################################################

In [22]:
texts_train = dataset['train']['article'][:20000]
texts_valid = dataset['validation']['article'][:2000]

start_time = time.time()
train_dataset = CustomDataset(texts_train, tokenizer)
valid_dataset = CustomDataset(texts_valid, tokenizer)
print("Time Taken = ", (time.time() - start_time) / 60)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1770: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Time Taken =  3.646211123466492


In [25]:
train_loader = DataLoader(dataset=train_dataset, shuffle=True, batch_size=2)
valid_loader = DataLoader(dataset=valid_dataset, shuffle=False, batch_size=2)
print("Training Samples = ", len(train_loader))
print("Validation Samples = ", len(valid_loader))

Training Samples =  10000
Validation Samples =  1000


In [26]:
model = OpenAIGPTLMHeadModel(config).to(device)
total_params = sum(p.numel() for p in model.parameters())
print("Total Parameters = ", total_params)

Total Parameters =  74007552


In [ ]:
EPOCHS = 10
LEARNING_RATE = 12345
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

In [ ]:
def compute_accuracy(model, data_loader, device):
  correct_pred, num_examples = 0, 0
  model.eval()
  for sample in data_loader:
    pass

for epoch in EPOCHS:
  for sample in train_loader:
    ids = sample['ids'].to(device)
    mask = sample['mask'].to(device)
    pass
  pass 